# Clasificador de noticias

En esta práctica, crearemos un clasificador de noticias utilizando las técnicas de procesamiento del lenguaje natural que hemos visto en clase, centrándose en la representación del texto.

Usaremos el `dataset` [AG News](https://www.kaggle.com/amananandrai/ag-news-classification-dataset) que contiene 1,000,000 noticias de 4 categorías diferentes.

## Dataset

Para cargar el conjunto de datos, usaremos la librería `datasets`. Esta librería nos permitirá cargar muchos conjuntos de datos diferentes de una manera simple.En este caso, cargaremos el conjunto de datos AG News.

## Preparación del dataset

Para instalar las librerías necesarias, ejecutaremos la siguiente celda.

Usaremos `pytorch` (una libreria de deep learning), `pipeline` (una libreria de tratamiento de datos), `scikit-learn` (una libreria de machine learning) y `transformers` (una libreria de modelos de lenguaje).

In [1]:
# Instalamos las librerías necesarias en las versiones correctas

%pip install --upgrade torch datasets scikit-learn transformers gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32

In [2]:
from datasets import load_dataset

# Cargamos el conjunto de datos. Se descargará y almacenará automáticamente en local.
# Este conjunto de datos contiene noticias de diferentes categorías. En este caso
# usaremos las categorías de mundo, deportes, negocios y ciencia ficción/tecnología.

dataset = load_dataset('ag_news')

dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [3]:
print(dataset['train'][0])

print(dataset['train'].features)

classes = dataset['train'].features["label"].names
classes

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
{'text': Value('string'), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'])}


['World', 'Sports', 'Business', 'Sci/Tech']

Automáticamente, la función `load` ha dividido el conjunto de datos en dos conjuntos: uno de train y uno para test. Para acceder a estos conjuntos, usaremos los atributos `train` i `test` del objeto `dataset`. Estos atributos son objetos `data.Dataset` que contiene los ejemplos y etiquetas del conjunto de capacitación y prueba. Para acceder a ejemplos y etiquetas, utilizaremos los atributos `data` y `label` del objeto `data.Dataset`.

In [4]:
# Separar el conjunto de datos en entrenamiento y test
ds_train = dataset['train']
ds_test = dataset['test']
# Veamos cuántos ejemplos hay en cada set
print('Número de ejemplos de train:', len(ds_train))
print('Número de ejemplos de test:', len(ds_test))

Número de ejemplos de train: 120000
Número de ejemplos de test: 7600


Imprimimos los primeros 5 ejemplos del conjunto de entrenamiento.Como podemos ver, cada ejemplo es una noticia y su etiqueta.

In [5]:
# Imprimimos los primeros 5 ejemplos del conjunto de entrenamiento
for w in ds_train.take(5):
    print(f"{w['label']} ({classes[w['label']]}) -> {w['text']}")

2 (Business) -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
2 (Business) -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
2 (Business) -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
2 (Business) -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
2 (Business) -> Oil prices soar to

## Tokenización

La representación del texto en un modelo de idioma requiere que el texto se convierta en números. Si queremos una representación de nivel de palabra, necesitamos hacer dos cosas:
* Utilizar un **tokenizador** para dividir el texto en **tokens**.
* Construir un **vocabulario** con estos tokens.

In [6]:
# Utilizamos el tokenizador de Bert (uno de los primeros modelos de lenguaje basados ​​en transformación) para tokenizar las oraciones

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

print(tokenizer.tokenize("-- Hello, how are you doing today?"))

# Podiamos ver el vocabulario de tokenización
vocab = tokenizer.get_vocab()
print(len(vocab))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

['-', '-', 'hello', ',', 'how', 'are', 'you', 'doing', 'today', '?']
30522


Usando el tokenizer, también podemos convertir nuestra cadena tokenizada en un conjunto de números:

In [7]:
tokenitzada = tokenizer.tokenize("-- Hello, how are you doing today?")

def encode(text):
    tk = tokenizer.tokenize(text)
    return tokenizer.convert_tokens_to_ids(tk)

print(encode("-- Hello, how are you doing today?"))

[1011, 1011, 7592, 1010, 2129, 2024, 2017, 2725, 2651, 1029]


## Representación del texto

Para entrenar un modelo de redes neuronales, necesitamos representar el texto como números. En esta práctica, usaremos la representación de la representación de la bolsa de palabras (BoW) que consiste en representar cada palabra como un número. Esta representación es muy simple y no tiene en cuenta el orden de las palabras o su semántica. Pero es una representación que funciona lo suficientemente bien en muchos casos.

## Representación de la bolsa de las palabras

Aunque el significado de las palabras no es fácil de deducir sin poder acceder al contexto, en algunos casos, la representación de la bolsa de palabras puede ser útil.Por ejemplo, en el texto de una noticia, la palabra `covid` puede ser un buen indicador de que las noticias hablan sobre Covid-19 y la palabra `snow` puede ser un buen indicador de que las noticias hablan sobre el tiempo atmosférico.

De las técnicas clásicas de vectorización de texto, la más simple es la representación de la bolsa de las palabras (BoW). En esta representación, cada palabra se representa como un número. Para convertir un texto en una representación de BoW, primero creamos un vector con tantos ceros como las palabras están en el vocabulario. Luego, para cada palabra del texto, aumentamos el valor de la posición correspondiente al vector por 1. Por ejemplo, si el texto es `this sentence is a test sentence`, el vector resultante seria `[1, 2, 1, 1, 0, 0, 0, 0, 0, 0, ...]`.

Si recordamos la representación one-hot, veremos que la representación de BoW es muy similar. La diferencia es que la representación one-hot será una serie de vectores con un solo 1 y el resto de los valores en 0. En cambio, la representación de BoW será un vector con tantas veces que aparezca cada palabra. Podemos considerar que la representación de BoW sería la suma de vectores únicos.

Por ejemplo, si el texto es `this sentence is a test sentence`, el vector one-hot de la primera palabra seria `[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]` y el vector one-hot de la segunda palabra seria `[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...]`. La representación de BoW sería la suma de estos dos vectores: `[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...]`.

Para generar una representación de BoW, usaremos esta técnica para convertir cada palabra en un vector único y luego agregar todos los vectores. Para hacer esto, usaremos la función `to_bow` que crearemos a continuación. Esta función recibe un texto y devuelve un vector con la representación BoW del texto.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)

vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

Para calcular el vector BoW de una noticia de nuestro dataset AG_NEWS, podemos usar la siguiente función:

In [9]:
import torch

len_vocab = len(vocab)

def to_bow(text, tamany_vocabulari=len_vocab):
    res = torch.zeros(tamany_vocabulari, dtype=torch.float32)

    for i in encode(text):
        if i<tamany_vocabulari:
            res[i] += 1
    return res

print(ds_train[0])
print(to_bow(ds_train[0]["text"]))

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}
tensor([0., 0., 0.,  ..., 0., 0., 0.])


### Entrenamiento de modelos de clasificación BoW

Nuestro primer modelo será un clasificador de noticias utilizando la representación de BoW. Para hacer esto, crearemos un modelo de redes neuronales con una capa de entrada con tantas neuronas como las palabras están en nuestro vocabulario y una capa de salida con tantas neuronas como categorías que hay en nuestro conjunto de datos.

#### Representación BoW

Primero necesitamos convertir el texto en la representación de BoW utilizando la función `to_bow` que hemos creado antes. Esta función recibe un texto y devuelve un vector con la representación BoW del texto.

En pytorch se utilizan los `DataLoaders`, para cargar los datos en lotes y convertirlos en tensores de Pytorch. Aprovecharemos esta funcionalidad para convertir los datos de BoW en tensores de PyTorch, Usando el parámetro `collate_fn` del `DataLoader` y proporcionando una función que convierta los datos textuales en tensores de BoW.

In [10]:
from torch.utils.data import DataLoader

def bowify(batch):
    '''
    Esta característica recibe una lista de noticias y devuelve un tensor con las etiquetas
    (vector de floats) y otro con las noticias codificadas como BoW (matriz de floats donde cada fila
    es un vector de BoW).
    '''

    # Las etiquetas son 0, 1, 2 o 3.
    # Usamos Longtensor porque son enteros.

    etiquetes = torch.LongTensor([noticia["label"] for noticia in batch])

    # La noticias son tensores de BoW
    noticies = torch.stack([to_bow(noticia["text"]) for noticia in batch])

    return (
            etiquetes,
            noticies
    )


train_loader = DataLoader(ds_train, batch_size=16, collate_fn=bowify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=bowify)

### Modelo de classificación

Ahora definamos una red neuronal clasificadora simple que contiene una capa lineal. El tamaño del vector de entrada es igual a `vocab_size`, y el tamaño de salida corresponde al número de clases (4). Debido a que estamos resolviendo la tarea de clasificación, la función de activación final es `LogSoftmax()`.

In [11]:
net = torch.nn.Sequential(
    torch.nn.Linear(len(vocab), 4),
    torch.nn.LogSoftmax(dim=1)
)

### Entrenamiento del modelo

Ahora definiremos el bucle de entrenamiento estándar de Pytorch. Debido a que nuestro conjunto de datos es bastante grande, con nuestro propósito de enseñanza, entrenaremos solo para una epoca, y a veces incluso por menos de una epoca (especificando el parámetro `epoch_size` nos permite limitar el entrenamiento). También informaremos la precisión del entrenamiento acumulado durante el entrenamiento; La frecuencia de notificación se especifica utilizando el parámetro `report_freq`.

Para entrenar el modelo, usaremos el optimizador `Adam`(ya que es uno de los optimizadores más utilizados) y la función de costo `CrossEntropyLoss` (ya que tenemos un problema de calificación con más de dos clases).

In [12]:
def train_epoch(
    net,
    dataloader,
    lr=0.01,
    optimizer=None,
    loss_fn=torch.nn.NLLLoss(),
    epoch_size=None,
    report_freq=200,
):
    # Si no se especifica un optimizador, usamos Adam
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)

    # Ponemos la red en modo de entrenamiento.Esto activa el comportamiento de las capas de DropOut, por ejemplo.
    net.train()

    # Inicializar las variables que nos servirán para calcular la precisión
    total_loss, acc, count, i = 0, 0, 0, 0

    # Iteremamos sobre el dataloader
    for labels, features in dataloader:

        # Ponemos los gradientes a cero
        optimizer.zero_grad()

        # calculamos la salida de la red
        out = net(features)

        # Calculamos la pérdida. Esta función ya se aplica a Softmax a la salida.
        loss = loss_fn(out, labels)  # cross_entropy(out,labels)

        # Propagamos la pérdida de regreso. Esto hará que se calculen los gradientes .
        loss.backward()

        # Actualizamos los pesos de la red. Esto toma un paso de optimización.
        optimizer.step()

        # Actualizamos variables para calcular la precisión.
        total_loss += loss

        # Calculamos la precisión. Para hacer esto, debemos convertir la salida de red en etiquetas.
        # La clase con la mayor probabilidad es la que predecimos como etiqueta.
        _, predicted = torch.max(out, 1)
        acc += (predicted == labels).sum()

        # Actualizamos el contador de muestras
        count += len(labels)

        # Mostramos la precisión cada report_freq muestras
        i += 1
        if i % report_freq == 0:
            print(f"{count}: acc={acc.item()/count}")

        # Si se especifica epoch_size y ya hemos procesado este número de muestras, dejamos el bucle.
        if epoch_size and count > epoch_size:
            break
    return total_loss.item() / count, acc.item() / count


train_epoch(net, train_loader, epoch_size=15000)

3200: acc=0.7403125
6400: acc=0.80359375
9600: acc=0.8258333333333333
12800: acc=0.841171875


(0.03097911379230556, 0.84761460554371)

El modelo ha logrado una precisión cercano a `0.85` en el conjunto de entrenamiento; Un número suficientemente aceptable considerando que hemos simplificado el problema para reducir el tiempo de ejecución del tutorial. En un caso real, usaríamos todas las noticias del conjunto de entrenamiento y el modelo sería más preciso.

## Representación de Word2Vec

La representación de Word2Vec es una representación ampliamente utilizada en el procesamiento del lenguaje natural. Esta representación tiene en cuenta el contexto de las palabras y permite operaciones con las palabras. Por ejemplo, si restamos la palabra vector `king` y sumamos el vector de la palabra `woman`, obtendremos un vector que será muy similar al vector de la plabra `queen`.

Para generar representación de Word2Vec, usaremos la librería `gensim`. Esta librería contiene muchos modelos de representación de palabras. En este caso usaremos el modelo `word2vec-google-news-300` que contiene la representación de Word2Vec de 3 millones de palabras y frases.

> La primera vez que esta celda se está ejecutando, la función `load` Descargará el modelo de 1.5GB. Esto puede tomar unos minutos.
> Esta función devuelve un objeto `KeyedVectors` que contiene la representación Word2Vec.

In [13]:
import gensim.downloader as api

w2v = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


Ahora podemos acceder a la representación de Word2Vec de cada palabra. Por ejemplo, para acceder a la representación de la palabra `king`, usaremos la función `get_vector` del objeto `KeyedVectors`.

In [14]:
w2v.get_vector('king')

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

También podemos acceder a las palabras más similares a una palabra. Por ejemplo, para acceder a las palabras más similares a la palabra `king`, usaremos la función `most_similar` del objeto `KeyedVectors`.

In [15]:
for w, p in w2v.most_similar('king'):
    print(f"{w} -> {p}")

kings -> 0.7138045430183411
queen -> 0.6510956883430481
monarch -> 0.6413194537162781
crown_prince -> 0.6204220056533813
prince -> 0.6159993410110474
sultan -> 0.5864824056625366
ruler -> 0.5797567367553711
princes -> 0.5646552443504333
Prince_Paras -> 0.5432944297790527
throne -> 0.5422105193138123


Lo más interesante de la representación de Word2Vec es que los vectores tienen una estructura matemática que nos permite realizar operaciones con las palabras. Por ejemplo, si restamos el vector de la palabra `man` al vector de la palabra `king` y sumamos el vector de la palabra `woman`, obtendremos un vector que será muy similar al vector de la palabra `queen`.

$$ KING - MAN + WOMAN = QUEEN $$

Para hacer esta operación, usaremos la función `most_similar` del objeto `KeyedVectors` y pasaremos los vectores de las palabras `king`, `woman` y `man`. Esta característica devolverá una lista con las palabras más similares al vector resultante.Como podemos ver, la palabra más similar es `queen`.

In [16]:
w2v.most_similar(positive=['king', 'woman'], negative=['man'])[0]

('queen', 0.7118193507194519)

### Clasificador de Word2Vec

Ahora crearemos un clasificador de noticias utilizando la representación Word2Vec. Primero tendremos que obtener la representación de cada palabra para convertir el texto en vectores. Luego agregaremos todos los vectores para obtener un vector para cada noticia. Este vector será la representación de las noticias.

Para convertir un texto en un vector, usaremos la función `to_w2v` que crearemos a continuación. Esta función recibe un texto y devuelve un vector con la representación Word2Vec del texto.

In [17]:
def to_w2v(text):
    res = torch.zeros(300, dtype=torch.float32)
    for word in text:
        if word in w2v:
            res += torch.tensor(w2v.get_vector(word))
    return res

print(to_w2v(ds_train[0]["text"]))

tensor([-17.0809,  11.0404,  -0.9337,  12.4042,  -6.2286,   3.0224, -10.0442,
         -8.5156,  -5.9407,   1.1501,  -3.8471,  -8.0006, -18.2444,   4.3982,
        -14.2061,  11.0110,  11.2352,  14.8521,  -2.5686,   2.8961, -22.3914,
         -3.2182,   9.7872,   0.3238,  -8.6214,   4.2367, -21.9348,   5.7704,
         -0.6942,  -1.7075,  -2.4800,   2.1805,  -7.0602, -12.3824, -11.6949,
          8.2563, -18.9995,  11.3932,  -7.3198,   7.3370,  -6.1129,  -3.6244,
          5.8519,   8.3060,   3.9137,  -1.8091,  -3.2730, -15.8203,  -9.6418,
          8.9092, -16.8270,  24.5614,  -2.5387,  21.7112,   6.0571,  14.3324,
        -17.4978, -12.2693,   1.1129, -15.9192, -12.1886,  -9.5650, -19.0873,
         -7.7948,  -4.9111, -18.4653, -10.2332,  11.3437,  -6.0452,   5.4705,
          3.7500,  -9.5068,   4.4747,  -0.2912,  -3.9221,   0.3543,  13.0927,
          2.3088,   3.5300, -11.2126, -14.8031,  -2.9008,  -3.4219,  -0.3365,
         13.8353,   7.0914,  -5.2219,  22.0132,   4.2657,   5.84

Como lo hicimos con la representación de BoW, usaremos el `DataLoaders` de PyTorch para convertir los datos en vectores Word2Vec en tensores Pytorch. Aprovecharemos el parámetro `collate_fn` del `DataLoader` para proporcionar una función que convierta los datos textuales en tensores Word2Vec.

In [18]:
def w2vify(batch):
    etiquetes = torch.LongTensor([noticia["label"] for noticia in batch])
    noticies = torch.stack([to_w2v(tokenizer.tokenize(noticia["text"])) for noticia in batch])
    return etiquetes, noticies

train_loader = DataLoader(ds_train, batch_size=16, collate_fn=w2vify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=w2vify)

### Modelo de clasificación

Ahora crearemos el modelo de clasificación usando Pytorch. Definiremos un modelo simple con una capa lineal. El tamaño del vector de entrada será `300` (el tamaño de la representación Word2Vec) y el tamaño de la salida será el número de clases (4). Como estamos resolviendo una tarea de clasificación, la función de activación final será `LogSoftmax()`.

In [19]:
net = torch.nn.Sequential(
    torch.nn.Linear(300, 4),
    torch.nn.LogSoftmax(dim=1)
)

Finalmente, entrenamos al modelo utilizando el mismo procedimiento que hemos realizado con la representación de BoW.

In [20]:
train_epoch(net, train_loader, epoch_size=15000)

3200: acc=0.73375
6400: acc=0.76828125
9600: acc=0.7792708333333334
12800: acc=0.792578125


(0.08423855258966051, 0.7985740938166311)

El resultado no es muy bueno. Esto se debe a que el modelo Word2Vec que utilizamos no tiene las palabras que aparecen en el conjunto de datos. Por ejemplo, si buscamos la palabra `covid`, veremos que no aparece en el modelo.

Para resolver este problema, tendremos que usar un modelo Word2Vec entrenado con las palabras del conjunto de datos. Pero esto es muy lento y no lo haremos en este tutorial.